<a href="https://colab.research.google.com/github/raheelam98/LangChain_Fundamentals/blob/main/langchain_course_jupyter/section_3_data_processing/part4_custom_search_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Custom Search Agent**

In [2]:
# Install the required packages:
%%capture --no-stderr
%pip install -U  langsmith # check
%pip install --quiet -U  langchain_google_genai langchain_core langchain langgraph   python-dotenv langsmith

In [3]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ice_breaker"

In [4]:
# API Keys
# Get the GEMINI API key from user data
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=gemini_api_key, temperature=0)

In [ ]:
from langchain_core.prompts import PromptTemplate

def lookup(name: str):
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0
    )
    template = """give the full name {name_of_person} I want you to get it me a link to their Linkedin profile page.
                  Your answer should contain only a URL"""

    prompt_template = PromptTemplate(
        template=template, input_variables=["name_of_person"]
    )

In [ ]:
linkedin_url = lookup(name="Eden Marco")
print(linkedin_url)